In [1]:
!pip install -U bitsandbytes
!pip install --upgrade transformers
import bitsandbytes
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 61.8 MB/s eta 0:00:0000:010:01
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1


In [2]:
model_id = "unsloth/gemma-7b-bnb-4bit"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(model_id, 
                                            device_map="auto",
                                            torch_dtype=torch.float16,
                                            quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/opt/conda/lib/python3.10/site-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.57G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [3]:
prompt = """Ты - помощник, который оцениват здоровье спринта. У тебя есть критерии: 
-       статусы объектов изменяются равномерно. Должны переходить из статуса: “К выполнению” -> “В работе” -> “Сделано”. Объекты не должны массово переводиться в статус “Сделано” в последний день спринта; 
-       параметр “К выполнению” - не должно составлять более 20% от общего объема;
-       параметр “Снято” - не должно составлять более 10% от общего объема;
-       бэклог не должен изменяться более чем на 20% после начала спринта.

User:
{user_input}
"""
user_input = "Привет, расскажи что такое ML"
prompt = prompt.format(user_input=user_input)

In [7]:
tokenized_prompt = tokenizer.encode(prompt, return_tensors="pt").to("cuda")

In [8]:
tokenized_prompt

tensor([[     2,  64748,    728,  33953,   8337, 235269,  27309, 163689,  33096,
         142603,    941,  10373,  29197, 235265,   7878,  44568,  17127,  29854,
         139260, 235292, 235248,    108, 235290,    144,   2429,   3987,   6857,
         140671, 210252,  13144,  83892, 113605,   1011, 235265,  20979, 235346,
          37614,   7389,  63216,   3777,  86496,   3686, 235292,   1080, 235518,
          34866, 103102, 235369,   3978,   1080, 235488,  62701, 235369,   3978,
           1080, 235475,  18319,   1011,   4474,  36020,  82843,   4396,   2087,
          51419, 127669,   1607,   7389,  16465,   7045,    975, 156119,   1080,
         235475,  18319,   1011, 235369,    975, 143193,  19888,    941,  10373,
          29197, 235289, 235248,    108, 235290,    144,  43294,  85512,   1080,
         235518,  34866, 103102, 235369,    728,   2087,  57706,  89058,   1691,
          23380, 235248, 235284, 235276, 235358,   3025, 151021, 212008, 235289,
            108, 235290,    

In [9]:
with torch.no_grad():
    output = model.generate(tokenized_prompt, max_new_tokens=64)

generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

print(generated_text)

Ты - помощник, который оцениват здоровье спринта. У тебя есть критерии: 
-       статусы объектов изменяются равномерно. Должны переходить из статуса: “К выполнению” -> “В работе” -> “Сделано”. Объекты не должны массово переводиться в статус “Сделано” в последний день спринта; 
-       параметр “К выполнению” - не должно составлять более 20% от общего объема;
-       параметр “Снято” - не должно составлять более 10% от общего объема;
-       бэклог не должен изменяться более чем на 20% после начала спринта.

User:
Привет, расскажи что такое ML
ML - Machine Learning
ML - это метод обучения машин
ML - это метод обучения машин

